In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
df = pd.read_csv('zomato.csv')
df.head()


,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775.0,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787.0,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918.0,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88.0,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166.0,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [ ]:

df.drop(['url', 'phone', 'dish_liked', 'menu_item'], axis=1, inplace=True)


df.drop_duplicates(inplace=True)


df.dropna(inplace=True)


df['rate'] = df['rate'].apply(lambda x: float(x.split('/')[0]) if '/' in str(x) else None)


df['approx_cost(for two people)'] = df['approx_cost(for two people)'].apply(lambda x: float(x.replace(',', '')) if ',' in str(x) else float(x))


df['features'] = df['cuisines'] + ' ' + df['rest_type'] + ' ' + df['reviews_list']

In [ ]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['features'].values.astype('U'))


In [ ]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)


indices = pd.Series(df.index, index=df['name']).drop_duplicates()

In [ ]:
def recommend(name):
    global cosine_similarities, indices, df


    if name not in indices:
        print(f"Restaurant '{name}' not found.")
        return None


    idx = indices[name]


    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)


    top10_indexes = list(score_series.iloc[1:11].index)


    return df.iloc[top10_indexes]


recommend('Fast And Fresh')

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,approx_cost(for two people),reviews_list,listed_in(type),listed_in(city),features
31,"2/1, 7th Main, Dwarakangar, Hosakeregalli, Ban...",Foodiction,Yes,No,2.8,506.0,Banashankari,Quick Bites,"North Indian, Fast Food, Chinese, Burger",500.0,"[('Rated 1.0', ""RATED\n Worst restaurant ever...",Delivery,Banashankari,"North Indian, Fast Food, Chinese, Burger Quick..."
308,"534, 49th Cross, 50 Feet Road, Kumaraswamy Lay...",Mast Punjabi,Yes,No,2.6,231.0,Kumaraswamy Layout,Quick Bites,"North Indian, Chinese, Rolls, Biryani, Kebab",400.0,"[('Rated 1.0', 'RATED\n Not tasty, no clean p...",Delivery,Banashankari,"North Indian, Chinese, Rolls, Biryani, Kebab Q..."
348,"22nd Cross, 22nd Main Road, 2nd Stage, Near BD...",Sri Krishna Sagar,Yes,No,3.2,119.0,Banashankari,Casual Dining,"North Indian, Chinese",400.0,"[('Rated 3.0', 'RATED\n Food was fine but was...",Delivery,Banashankari,"North Indian, Chinese Casual Dining [('Rated 3..."
345,"27th Cross Rd, Banashankari Stage II, Banashan...",Coffee Tindi,Yes,No,3.8,75.0,Banashankari,"Cafe, Quick Bites","Cafe, South Indian",200.0,"[('Rated 5.0', 'RATED\n please provide some e...",Delivery,Banashankari,"Cafe, South Indian Cafe, Quick Bites [('Rated ..."
26,"27th Cross Rd, Banashankari Stage II, Banashan...",Coffee Tindi,Yes,No,3.8,75.0,Banashankari,"Cafe, Quick Bites","Cafe, South Indian",200.0,"[('Rated 5.0', 'RATED\n please provide some e...",Cafes,Banashankari,"Cafe, South Indian Cafe, Quick Bites [('Rated ..."
376,"84/1, South End Road, Basavanagudi, Bangalore",Nammura Donne Biriyani,Yes,No,3.5,155.0,Basavanagudi,Quick Bites,"Biryani, Fast Food",400.0,"[('Rated 2.0', ""RATED\n taste is average , le...",Delivery,Banashankari,"Biryani, Fast Food Quick Bites [('Rated 2.0', ..."
272,"1349, 4th Cross Road, Kilkodungalur, RPC Layou...",Bendakaluru Bytes,Yes,No,3.2,154.0,Vijay Nagar,Quick Bites,Fast Food,300.0,"[('Rated 1.0', 'RATED\n Food quality is not g...",Delivery,Banashankari,"Fast Food Quick Bites [('Rated 1.0', 'RATED\n ..."
302,"78, Nagendra Block, Giri Nagar, II Phase, Bana...",Vasanth Vihar - Since 1965,Yes,No,3.6,95.0,Banashankari,Quick Bites,"South Indian, Street Food",150.0,"[('Rated 1.0', 'RATED\n lack of quantity and ...",Delivery,Banashankari,"South Indian, Street Food Quick Bites [('Rated..."
266,"433/30/1, 10th Main, 28th A cross, 4th Block, ...",Anda Ka Funda,Yes,No,3.4,89.0,Jayanagar,Quick Bites,"North Indian, Chinese, Rolls",250.0,"[('Rated 1.0', 'RATED\n its not at all good')...",Delivery,Banashankari,"North Indian, Chinese, Rolls Quick Bites [('Ra..."
159,"28/29, 9th Main Road, 3rd Block, Jayanagar, Ba...",Wahab,Yes,No,3.7,97.0,Jayanagar,Quick Bites,"North Indian, Mughlai, Chinese",350.0,"[('Rated 3.0', 'RATED\n Ordered chicken kabab...",Delivery,Banashankari,"North Indian, Mughlai, Chinese Quick Bites [('..."
